<a href="https://colab.research.google.com/github/ruslanbulat21-svg/Test_task_yolo/blob/main/YOLO11s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics roboflow --quiet

import zipfile
import os

with zipfile.ZipFile('/content/drive/MyDrive/Pens.v2i.yolov11.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

print("Содержимое датасета:")
!find /content/dataset -type f -name "*.yaml" -o -name "*.yml" | head -5

Содержимое датасета:
/content/dataset/data.yaml


In [ ]:
!uv pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.4/112.6 GB disk)


In [ ]:
import glob

yaml_files = glob.glob('/content/dataset/**/*.yaml', recursive=True) + glob.glob('/content/dataset/**/*.yml', recursive=True)

print("Найдены YAML файлы:")
for file in yaml_files:
    print(f"- {file}")


if yaml_files:
    with open(yaml_files[0], 'r') as f:
        print("\nСодержимое конфиг файла:")
        print(f.read())

Найдены YAML файлы:
- /content/dataset/data.yaml

Содержимое конфиг файла:
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['Pen', 'Pencil']

roboflow:
  workspace: ruslan-6lbtj
  project: pens-t4yft
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/ruslan-6lbtj/pens-t4yft/dataset/2


In [ ]:
import torch
print(f"Доступен ли GPU: {torch.cuda.is_available()}")
print(f"Количество GPU: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")

Доступен ли GPU: True
Количество GPU: 1
Название GPU: Tesla T4


In [ ]:
from ultralytics import YOLO
import os


model = YOLO('yolo11s.pt')


results = model.train(
    data="/content/dataset/data.yaml",  # путь к конфигурационному файлу
    epochs=50,         # количество эпох
    imgsz=640,         # размер изображения
    batch=16,          # размер батча
    lr0=0.01,          # скорость обучения
    device=0,          # использование GPU
    workers=4,         # количество workers
    patience=10,       # ранняя остановка
    save=True,         # сохранение чекпоинтов
    pretrained=True,   # использование предобученных весов
    optimizer='auto',  # автоматический выбор оптимизатора
    seed=42            # для воспроизводимости
)

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, p

In [ ]:
results = model.predict('/content/dataset/valid/images', save=True, conf=0.5)

print("✅ Предсказания сохранены в /content/runs/detect/predict yolo n/")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

result_images = glob.glob('/content/runs/detect/predict yolo n/*.jpg')[:3]

if result_images:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    for i, img_path in enumerate(result_images[:3]):
        img = mpimg.imread(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(f'Результат {i+1}')
    plt.tight_layout()
    plt.show()


image 1/10 /content/dataset/valid/images/IMG_20251031_154154_jpg.rf.6b9c012d98c1cc8e6a58300361fc0986.jpg: 640x480 5 Pens, 3 Pencils, 13.2ms
image 2/10 /content/dataset/valid/images/IMG_20251031_154157_jpg.rf.5726851633fd7ece5f0135ea4aa0b526.jpg: 640x480 3 Pens, 6 Pencils, 12.4ms
image 3/10 /content/dataset/valid/images/IMG_20251031_154423_jpg.rf.ddab82baaf67c82aaa925c0911ee20db.jpg: 480x640 3 Pens, 3 Pencils, 15.6ms
image 4/10 /content/dataset/valid/images/IMG_20251031_154610_jpg.rf.6c742245c6aa4b01ccdf166be5d06197.jpg: 640x480 3 Pens, 2 Pencils, 13.6ms
image 5/10 /content/dataset/valid/images/IMG_20251031_154626_jpg.rf.0f9ba48e1b074aeaa4a52d1d034fd209.jpg: 480x640 2 Pens, 4 Pencils, 13.4ms
image 6/10 /content/dataset/valid/images/VID_20251031_154851_mp4-0054_jpg.rf.db12e3a6558c556acf9a837721bb2ec3.jpg: 640x384 5 Pens, 2 Pencils, 11.9ms
image 7/10 /content/dataset/valid/images/VID_20251031_154851_mp4-0087_jpg.rf.78d63fb00194184d2a91a88e210142ec.jpg: 640x384 5 Pens, 1 Pencil, 10.9ms
im

In [ ]:
from google.colab.patches import cv2_imshow
from google.colab import output
from IPython.display import display, Javascript
import cv2

In [ ]:
import cv2
import os
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import glob
from IPython.display import display, HTML
import base64

model = YOLO('/content/runs/detect/train/weights/best.pt')

video_folder = '/content/dataset/valid/videos'
output_folder = '/content/runs/detect/video_results'

os.makedirs(output_folder, exist_ok=True)

video_files = glob.glob(os.path.join(video_folder, '*'))
video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv']
video_files = [f for f in video_files if os.path.splitext(f)[1].lower() in video_extensions]

print(f"Найдено видео файлов: {len(video_files)}")

for video_path in video_files:
    print(f"Обрабатывается видео: {os.path.basename(video_path)}")

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    output_path = os.path.join(output_folder, f"result_{os.path.basename(video_path)}")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    processed_frames = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if frame_count % 2 == 0:
            results = model.predict(frame, conf=0.5, verbose=False)
            annotated_frame = results[0].plot()
            out.write(annotated_frame)
            processed_frames += 1
        else:
            out.write(frame)

        if frame_count % 100 == 0:
            print(f"Обработано кадров: {frame_count}/{total_frames}")

    cap.release()
    out.release()

    print(f"Видео обработано: {os.path.basename(video_path)}")

    with open(output_path, 'rb') as f:
        video_bytes = f.read()
    video_b64 = base64.b64encode(video_bytes).decode()
    video_html = f'''
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
    </video>
    <p>Файл: {os.path.basename(video_path)}</p>
    '''
    display(HTML(video_html))

print(f"Все видео обработаны! Результаты в папке: {output_folder}")